In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow_datasets as tfds
import IPython.display as disp

!pip install geemap
import geemap



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.0 MB/s eta 0:00:00


In [4]:
import ee
ee.Authenticate()
ee.Initialize(project="federicodeguio")

In [6]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
%cd /content/drive/My Drive/Didattica/Corsi/AI/Lectures/Lecture16+17/TerrainClassification/

%ls

Mounted at /content/drive/
/content/drive/My Drive/Didattica/Corsi/AI/Lectures/Lecture16+17/TerrainClassification
eurosat_model.png  log1/             scripts/
gee/               log2/             select_area_with_geoJSON.ipynb
GEE/               s2cnnclass.ipynb  Visualize_and_classify.ipynb


In [7]:
geometry = ee.Geometry.Point([9.269243, 45.4654219])
roi = geometry.buffer(25000)


collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
                .filterBounds(geometry) \
                .filterDate('2022-01-01', '2023-04-10') \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',10)) \
                .sort("CLOUDY_PIXEL_PERCENTAGE") \
                .select('B2','B3','B4')


print(collection.size().getInfo())


29


In [8]:
collection.aggregate_array('CLOUDY_PIXEL_PERCENTAGE').getInfo()


[0.001712,
 0.112465,
 0.13278,
 0.33478,
 0.367482,
 0.401551,
 0.860038,
 0.90571,
 0.942787,
 0.954347,
 1.215958,
 1.336731,
 1.35251,
 1.472821,
 1.958619,
 2.017863,
 2.932207,
 3.022407,
 3.103338,
 3.621845,
 3.632055,
 3.676591,
 4.226599,
 4.480009,
 4.785138,
 5.860298,
 7.695011,
 8.567567,
 9.850558]

In [9]:
imageList = collection.toList(collection.size())

image = imageList.get(0)
image = ee.Image(image)

In [10]:
Map = geemap.Map()
vis_param = {'min': 0,
             'max': 3000,
             'bands': ['B4', 'B3', 'B2'],
             'gamma': 1.5}

Map.addLayer(image, vis_param, "First image")
Map.centerObject(geometry, 13)

Map

Map(center=[45.465421899999996, 9.269243000000001], controls=(WidgetControl(options=['position', 'transparent_…

In [13]:
gdexport = ee.batch.Export.image.toDrive(image.clip(roi),
                                         description='driveExportTask_s2',
                                         folder = 'gee',
                                         fileNamePrefix = 's2_Milan_test_big',
                                         scale = 10,
                                         maxPixels = 1e11)
gdexport.start()

In [21]:
gdexport.status()

{'state': 'COMPLETED',
 'description': 'driveExportTask_s2',
 'priority': 100,
 'creation_timestamp_ms': 1715070417429,
 'update_timestamp_ms': 1715070817672,
 'start_timestamp_ms': 1715070419918,
 'task_type': 'EXPORT_IMAGE',
 'destination_uris': ['https://drive.google.com/#folders/1-71Ll9e-81azqHfme6d-IME2LfW1-R3t'],
 'attempt': 1,
 'batch_eecu_usage_seconds': 56.84774398803711,
 'id': 'Y655OKZKGUCRUC2SNFWHPJPM',
 'name': 'projects/federicodeguio/operations/Y655OKZKGUCRUC2SNFWHPJPM'}

In [18]:
run ./scripts/dispms -f ./GEE/s2_Milan_test_big.tif -e 4 -p [1,2,3] -d [600,600,3000,3000]

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
run ./scripts/S2cnnclassify -d [600,600,3000,3000] -s 8 ./GEE/s2_Milan_test_big.tif

Classifying...:  25%|██▌       | 95/375 [44:49<2:39:47, 34.24s/it]